In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np
from pylab import rcParams

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_recall_curve
from sklearn.metrics import recall_score, classification_report, auc, roc_curve
from sklearn.metrics import precision_recall_fscore_support, f1_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from numpy import mean
from numpy import std

from sklearn.model_selection import train_test_split

SEED = 123 #used to help randomly select the data points
DATA_SPLIT_PCT = 0.2

rcParams['figure.figsize'] = 8, 6
LABELS = ["Normal","Break"]

In [2]:
df = pd.read_csv("bosch_selected_without_null.csv") 
df

,Unnamed: 0,L1_S24_F1622,L1_S24_F1723,L1_S24_F1166,L1_S24_F1846,L0_S3_F96,L3_S33_F3859,L3_S33_F3857,L1_S24_F1844,L1_S24_F1609,...,L1_S25_F2237,L3_S30_F3574,L3_S30_F3769,L1_S25_F2925,L1_S25_F2484,L3_S34_F3880,L3_S41_F4020,L0_S14_F370,L2_S26_F3113,L1_S24_F844
0,0,0.000018,0.000292,0.001569,-0.006682,-0.000358,-0.048,-0.102,0.001034,0.005514,...,0.035576,0.001711,-0.007,0.001233,0.016188,0.0,0.008414,-0.005228,-0.001004,-0.019477
1,1,0.000018,0.000292,0.001569,-0.006682,-0.000515,-0.210,-0.026,0.001034,0.005514,...,0.035576,0.033000,-0.024,0.001233,0.016188,0.0,0.008414,-0.324000,-0.001004,-0.019477
2,2,0.000018,0.000292,0.001569,-0.006682,-0.000515,0.236,0.191,0.001034,0.005514,...,0.035576,-0.147000,-0.026,0.001233,0.016188,0.0,0.008414,-0.324000,-0.001011,-0.019477
3,3,0.000018,0.000292,0.001569,-0.006682,-0.000515,0.165,0.031,0.001034,0.005514,...,0.035576,-0.054000,0.022,0.001233,0.016188,0.0,0.008414,-0.324000,-0.001011,-0.019477
4,4,0.000018,0.000292,0.001569,-0.006682,0.311000,0.147,-0.049,0.001034,0.005514,...,0.035576,0.119000,0.022,0.001233,0.016188,0.0,0.008746,-0.324000,-0.001011,-0.019477
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1183742,1183742,-0.000142,-0.005846,-0.014684,0.004028,-0.005042,0.152,-0.095,0.002243,0.000259,...,0.045373,-0.168000,-0.043,-0.005163,-0.025226,0.0,0.019205,-0.019136,-0.108000,0.008068
1183743,1183743,-0.000142,-0.005846,-0.014684,0.004028,-0.005195,-0.139,-0.184,0.002243,0.000259,...,0.045373,0.141000,-0.004,-0.005163,-0.025226,0.0,0.019205,-0.019136,0.007303,0.008068
1183744,1183744,-0.000142,-0.005846,-0.014684,0.004028,-0.245000,-0.114,0.019,0.002243,0.000259,...,0.045373,-0.154000,-0.001,-0.005163,-0.025226,0.0,0.019205,-0.018788,0.007303,0.008068
1183745,1183745,-0.000142,-0.005846,-0.014684,0.004028,-0.256000,0.031,0.072,0.002243,0.000259,...,0.045373,0.119000,-0.001,-0.005163,-0.025226,0.0,0.019205,-0.018788,0.007303,0.008068


# without curve-shifting

In [3]:
df = df.drop(['Unnamed: 0'], axis=1)

In [4]:
df_response= pd.read_csv('bosch_response.csv')
df_response = df_response.rename(columns={'0.1': 'Response'})
df_response
df_response = df_response.drop(['0'], axis=1)
df_response

,Response
0,0
1,0
2,0
3,0
4,0
...,...
1183741,0
1183742,0
1183743,0
1183744,0


In [5]:
df_new=df.join(df_response)
df_new

,L1_S24_F1622,L1_S24_F1723,L1_S24_F1166,L1_S24_F1846,L0_S3_F96,L3_S33_F3859,L3_S33_F3857,L1_S24_F1844,L1_S24_F1609,L1_S24_F1474,...,L3_S30_F3574,L3_S30_F3769,L1_S25_F2925,L1_S25_F2484,L3_S34_F3880,L3_S41_F4020,L0_S14_F370,L2_S26_F3113,L1_S24_F844,Response
0,0.000018,0.000292,0.001569,-0.006682,-0.000358,-0.048,-0.102,0.001034,0.005514,0.001974,...,0.001711,-0.007,0.001233,0.016188,0.0,0.008414,-0.005228,-0.001004,-0.019477,0.0
1,0.000018,0.000292,0.001569,-0.006682,-0.000515,-0.210,-0.026,0.001034,0.005514,0.001974,...,0.033000,-0.024,0.001233,0.016188,0.0,0.008414,-0.324000,-0.001004,-0.019477,0.0
2,0.000018,0.000292,0.001569,-0.006682,-0.000515,0.236,0.191,0.001034,0.005514,0.001974,...,-0.147000,-0.026,0.001233,0.016188,0.0,0.008414,-0.324000,-0.001011,-0.019477,0.0
3,0.000018,0.000292,0.001569,-0.006682,-0.000515,0.165,0.031,0.001034,0.005514,0.001974,...,-0.054000,0.022,0.001233,0.016188,0.0,0.008414,-0.324000,-0.001011,-0.019477,0.0
4,0.000018,0.000292,0.001569,-0.006682,0.311000,0.147,-0.049,0.001034,0.005514,0.001974,...,0.119000,0.022,0.001233,0.016188,0.0,0.008746,-0.324000,-0.001011,-0.019477,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1183742,-0.000142,-0.005846,-0.014684,0.004028,-0.005042,0.152,-0.095,0.002243,0.000259,-0.018253,...,-0.168000,-0.043,-0.005163,-0.025226,0.0,0.019205,-0.019136,-0.108000,0.008068,0.0
1183743,-0.000142,-0.005846,-0.014684,0.004028,-0.005195,-0.139,-0.184,0.002243,0.000259,-0.018253,...,0.141000,-0.004,-0.005163,-0.025226,0.0,0.019205,-0.019136,0.007303,0.008068,0.0
1183744,-0.000142,-0.005846,-0.014684,0.004028,-0.245000,-0.114,0.019,0.002243,0.000259,-0.018253,...,-0.154000,-0.001,-0.005163,-0.025226,0.0,0.019205,-0.018788,0.007303,0.008068,0.0
1183745,-0.000142,-0.005846,-0.014684,0.004028,-0.256000,0.031,0.072,0.002243,0.000259,-0.018253,...,0.119000,-0.001,-0.005163,-0.025226,0.0,0.019205,-0.018788,0.007303,0.008068,0.0


In [6]:
df_new = df_new[df_new['Response'].notna()]
df_new

,L1_S24_F1622,L1_S24_F1723,L1_S24_F1166,L1_S24_F1846,L0_S3_F96,L3_S33_F3859,L3_S33_F3857,L1_S24_F1844,L1_S24_F1609,L1_S24_F1474,...,L3_S30_F3574,L3_S30_F3769,L1_S25_F2925,L1_S25_F2484,L3_S34_F3880,L3_S41_F4020,L0_S14_F370,L2_S26_F3113,L1_S24_F844,Response
0,0.000018,0.000292,0.001569,-0.006682,-0.000358,-0.048,-0.102,0.001034,0.005514,0.001974,...,0.001711,-0.007,0.001233,0.016188,0.0,0.008414,-0.005228,-0.001004,-0.019477,0.0
1,0.000018,0.000292,0.001569,-0.006682,-0.000515,-0.210,-0.026,0.001034,0.005514,0.001974,...,0.033000,-0.024,0.001233,0.016188,0.0,0.008414,-0.324000,-0.001004,-0.019477,0.0
2,0.000018,0.000292,0.001569,-0.006682,-0.000515,0.236,0.191,0.001034,0.005514,0.001974,...,-0.147000,-0.026,0.001233,0.016188,0.0,0.008414,-0.324000,-0.001011,-0.019477,0.0
3,0.000018,0.000292,0.001569,-0.006682,-0.000515,0.165,0.031,0.001034,0.005514,0.001974,...,-0.054000,0.022,0.001233,0.016188,0.0,0.008414,-0.324000,-0.001011,-0.019477,0.0
4,0.000018,0.000292,0.001569,-0.006682,0.311000,0.147,-0.049,0.001034,0.005514,0.001974,...,0.119000,0.022,0.001233,0.016188,0.0,0.008746,-0.324000,-0.001011,-0.019477,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1183741,-0.000142,-0.038000,-0.014684,0.058000,-0.005042,0.054,0.033,0.261000,0.000259,-0.018253,...,0.032000,0.033,-0.005163,-0.025226,0.0,0.019205,-0.019136,0.007360,0.008068,0.0
1183742,-0.000142,-0.005846,-0.014684,0.004028,-0.005042,0.152,-0.095,0.002243,0.000259,-0.018253,...,-0.168000,-0.043,-0.005163,-0.025226,0.0,0.019205,-0.019136,-0.108000,0.008068,0.0
1183743,-0.000142,-0.005846,-0.014684,0.004028,-0.005195,-0.139,-0.184,0.002243,0.000259,-0.018253,...,0.141000,-0.004,-0.005163,-0.025226,0.0,0.019205,-0.019136,0.007303,0.008068,0.0
1183744,-0.000142,-0.005846,-0.014684,0.004028,-0.245000,-0.114,0.019,0.002243,0.000259,-0.018253,...,-0.154000,-0.001,-0.005163,-0.025226,0.0,0.019205,-0.018788,0.007303,0.008068,0.0


In [7]:
df_new = df_new[np.isfinite(df_new).all(1)]
df_new

,L1_S24_F1622,L1_S24_F1723,L1_S24_F1166,L1_S24_F1846,L0_S3_F96,L3_S33_F3859,L3_S33_F3857,L1_S24_F1844,L1_S24_F1609,L1_S24_F1474,...,L3_S30_F3574,L3_S30_F3769,L1_S25_F2925,L1_S25_F2484,L3_S34_F3880,L3_S41_F4020,L0_S14_F370,L2_S26_F3113,L1_S24_F844,Response
0,0.000018,0.000292,0.001569,-0.006682,-0.000358,-0.048,-0.102,0.001034,0.005514,0.001974,...,0.001711,-0.007,0.001233,0.016188,0.0,0.008414,-0.005228,-0.001004,-0.019477,0.0
1,0.000018,0.000292,0.001569,-0.006682,-0.000515,-0.210,-0.026,0.001034,0.005514,0.001974,...,0.033000,-0.024,0.001233,0.016188,0.0,0.008414,-0.324000,-0.001004,-0.019477,0.0
2,0.000018,0.000292,0.001569,-0.006682,-0.000515,0.236,0.191,0.001034,0.005514,0.001974,...,-0.147000,-0.026,0.001233,0.016188,0.0,0.008414,-0.324000,-0.001011,-0.019477,0.0
3,0.000018,0.000292,0.001569,-0.006682,-0.000515,0.165,0.031,0.001034,0.005514,0.001974,...,-0.054000,0.022,0.001233,0.016188,0.0,0.008414,-0.324000,-0.001011,-0.019477,0.0
4,0.000018,0.000292,0.001569,-0.006682,0.311000,0.147,-0.049,0.001034,0.005514,0.001974,...,0.119000,0.022,0.001233,0.016188,0.0,0.008746,-0.324000,-0.001011,-0.019477,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1183741,-0.000142,-0.038000,-0.014684,0.058000,-0.005042,0.054,0.033,0.261000,0.000259,-0.018253,...,0.032000,0.033,-0.005163,-0.025226,0.0,0.019205,-0.019136,0.007360,0.008068,0.0
1183742,-0.000142,-0.005846,-0.014684,0.004028,-0.005042,0.152,-0.095,0.002243,0.000259,-0.018253,...,-0.168000,-0.043,-0.005163,-0.025226,0.0,0.019205,-0.019136,-0.108000,0.008068,0.0
1183743,-0.000142,-0.005846,-0.014684,0.004028,-0.005195,-0.139,-0.184,0.002243,0.000259,-0.018253,...,0.141000,-0.004,-0.005163,-0.025226,0.0,0.019205,-0.019136,0.007303,0.008068,0.0
1183744,-0.000142,-0.005846,-0.014684,0.004028,-0.245000,-0.114,0.019,0.002243,0.000259,-0.018253,...,-0.154000,-0.001,-0.005163,-0.025226,0.0,0.019205,-0.018788,0.007303,0.008068,0.0


In [8]:
X = df_new.drop(['Response'], axis=1)
y = df_new['Response']

X
X=X.fillna(0)
y=y.fillna(0)

In [9]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
X_train.shape, X_test.shape

((946996, 58), (236750, 58))

In [10]:
# Feature scaling

In [11]:
cols = X_train.columns

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train = pd.DataFrame(X_train, columns=[cols])
X_test = pd.DataFrame(X_test, columns=[cols])


In [13]:
# define model
model = AdaBoostClassifier()

In [14]:
clf_0 = model.fit(X_train, y_train)

In [15]:
pred_y_0 = clf_0.predict(X_test)

print(accuracy_score(pred_y_0, y_test))

0.994272439281943


In [16]:
target_names = ['class 0', 'class 1']
print(classification_report(y_test, pred_y_0, target_names=target_names))

confusion_matrix(y_test, pred_y_0)

              precision    recall  f1-score   support

     class 0       0.99      1.00      1.00    235396
     class 1       0.25      0.00      0.00      1354

    accuracy                           0.99    236750
   macro avg       0.62      0.50      0.50    236750
weighted avg       0.99      0.99      0.99    236750



array([[235393,      3],
       [  1353,      1]])